In [1]:
# Fabric Notebook: Gold Layer - DBT-Style Business Models
# Purpose: Create fact tables, dimensions, and KPIs with tests and documentation

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, sum as _sum, avg, count, countDistinct, round as spark_round,
    when, datediff, year, month, dayofmonth, current_timestamp, lit,
    concat_ws, coalesce, row_number
)
from pyspark.sql.window import Window
from datetime import datetime

print("=" * 80)
print("🏆 GOLD LAYER - DBT BUSINESS MODELS")
print("=" * 80)

# ============================================================================
# SETUP: READ SILVER DATA
# ============================================================================
print("\n📂 Reading Silver tables...")

df_silver = spark.read.table("silver_hotel_bookings")
df_weather = spark.read.table("silver_weather_data")
df_currency = spark.read.table("silver_currency_rates")

print(f"✅ Silver bookings: {df_silver.count():,} records")
print(f"✅ Weather data: {df_weather.count()} records")
print(f"✅ Currency data: {df_currency.count()} records")

# ============================================================================
# MODEL 1: DIM_HOTEL (Hotel Dimension)
# ============================================================================
print("\n" + "=" * 80)
print("📊 MODEL 1: DIM_HOTEL (Hotel Dimension)")
print("=" * 80)

print("\n🔨 Building dim_hotel...")

dim_hotel = df_silver.select(
    col("hotel_id"),
    col("hotel_name"),
    col("country"),
    col("city"),
    col("hotel_type"),
    col("star_rating"),
    col("total_rooms"),
    col("hotel_facilities"),
    col("latitude"),
    col("longitude")
).distinct()

# Add metadata
dim_hotel = dim_hotel \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .withColumn("model_version", lit("v1.0")) \
    .withColumn("source_model", lit("silver_hotel_bookings"))

print(f"✅ dim_hotel created: {dim_hotel.count():,} unique hotels")

# DATA QUALITY TESTS for dim_hotel
print("\n🧪 Running tests for dim_hotel...")

# Test 1: hotel_id should be unique
test1_duplicates = dim_hotel.count() - dim_hotel.select("hotel_id").distinct().count()
if test1_duplicates == 0:
    print("   ✅ TEST PASSED: hotel_id is unique")
else:
    print(f"   ❌ TEST FAILED: {test1_duplicates} duplicate hotel_ids")

# Test 2: hotel_id should not be null
test2_nulls = dim_hotel.filter(col("hotel_id").isNull()).count()
if test2_nulls == 0:
    print("   ✅ TEST PASSED: hotel_id has no nulls")
else:
    print(f"   ❌ TEST FAILED: {test2_nulls} null hotel_ids")

# Test 3: hotel_name should not be null
test3_nulls = dim_hotel.filter(col("hotel_name").isNull()).count()
if test3_nulls == 0:
    print("   ✅ TEST PASSED: hotel_name has no nulls")
else:
    print(f"   ❌ TEST FAILED: {test3_nulls} null hotel_names")

# Write to Gold
print("\n💾 Writing dim_hotel to Gold layer...")
dim_hotel.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_dim_hotel")

print("✅ dim_hotel saved successfully")

# ============================================================================
# MODEL 2: DIM_CUSTOMER (Customer Dimension)
# ============================================================================
print("\n" + "=" * 80)
print("📊 MODEL 2: DIM_CUSTOMER (Customer Dimension)")
print("=" * 80)

print("\n🔨 Building dim_customer...")

dim_customer = df_silver.select(
    col("customer_id"),
    col("first_name"),
    col("last_name"),
    col("full_name"),
    col("email"),
    col("phone"),
    col("birth_date"),
    col("gender"),
    col("country_customer").alias("country"),
    col("city_customer").alias("city"),
    col("language_preference"),
    col("loyalty_level_customer").alias("loyalty_level"),
    col("registration_date"),
    col("marketing_consent")
).distinct()

# Add derived columns
dim_customer = dim_customer \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .withColumn("model_version", lit("v1.0")) \
    .withColumn("source_model", lit("silver_hotel_bookings")) \
    .withColumn("customer_age", 
        when(col("birth_date").isNotNull(), 
             year(current_timestamp()) - year(col("birth_date"))
        ).otherwise(None)
    )

print(f"✅ dim_customer created: {dim_customer.count():,} unique customers")

# DATA QUALITY TESTS for dim_customer
print("\n🧪 Running tests for dim_customer...")

# Test 1: customer_id should be unique
test1_duplicates = dim_customer.count() - dim_customer.select("customer_id").distinct().count()
if test1_duplicates == 0:
    print("   ✅ TEST PASSED: customer_id is unique")
else:
    print(f"   ❌ TEST FAILED: {test1_duplicates} duplicate customer_ids")

# Test 2: customer_id should not be null
test2_nulls = dim_customer.filter(col("customer_id").isNull()).count()
if test2_nulls == 0:
    print("   ✅ TEST PASSED: customer_id has no nulls")
else:
    print(f"   ❌ TEST FAILED: {test2_nulls} null customer_ids")

# Test 3: email should be valid format (contains @)
test3_invalid = dim_customer.filter(
    col("email").isNotNull() & ~col("email").contains("@")
).count()
if test3_invalid == 0:
    print("   ✅ TEST PASSED: all emails are valid format")
else:
    print(f"   ❌ TEST FAILED: {test3_invalid} invalid emails")

# Write to Gold
print("\n💾 Writing dim_customer to Gold layer...")
dim_customer.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_dim_customer")

print("✅ dim_customer saved successfully")

# ============================================================================
# MODEL 3: FACT_BOOKING (Fact Table) - Basitleştirilmiş Son Düzeltme
# ============================================================================
print("\n" + "=" * 80)
print("📊 MODEL 3: FACT_BOOKING (Fact Table)")
print("=" * 80)

print("\n🔨 Building fact_booking...")

# Select only valid bookings with required fields
fact_booking = df_silver.filter(
    col("booking_id").isNotNull() &
    col("hotel_id").isNotNull() &
    col("customer_id").isNotNull() &
    col("booking_date").isNotNull()
)

# Select fact columns
fact_booking = fact_booking.select(
    # Keys
    col("booking_id"),
    col("hotel_id"),
    col("customer_id"),
    
    # Dates
    col("booking_date"),
    col("checkin_date"),
    col("checkout_date"),
    col("cancellation_date"),
    
    # Booking details
    col("nights"),
    col("adults"),
    col("children"),
    col("infants"),
    col("room_type"), 
    col("rooms_booked"),
    col("booking_channel"),
    col("booking_source"),
    
    # Financial metrics
    col("total_price"),
    col("room_price"),
    col("tax_amount"),
    col("service_fee"),
    col("paid_amount"),
    col("discount_amount"),
    
    # Status
    col("is_cancelled"),
    col("payment_status"), 
    col("booking_status") 
)

# Add derived metrics (total_guests sütunu yeniden hesaplanarak oluşturuldu)
fact_booking = fact_booking \
    .withColumn("total_guests", 
        coalesce(col("adults"), lit(0)) + 
        coalesce(col("children"), lit(0)) + 
        coalesce(col("infants"), lit(0))
    ) \
    .withColumn("revenue_per_night",
        when(col("nights") > 0, col("total_price") / col("nights"))
        .otherwise(None)
    ) \
    .withColumn("revenue_per_guest",
        when(col("total_guests") > 0, col("total_price") / col("total_guests"))
        .otherwise(None)
    ) \
    .withColumn("booking_year", year(col("booking_date"))) \
    .withColumn("booking_month", month(col("booking_date"))) \
    .withColumn("checkin_year", year(col("checkin_date"))) \
    .withColumn("checkin_month", month(col("checkin_date"))) \
    .withColumn("lead_time_days",
        when(col("booking_date").isNotNull() & col("checkin_date").isNotNull(),
            datediff(col("checkin_date"), col("booking_date"))
        ).otherwise(None)
    )

# Add Gold metadata
fact_booking = fact_booking \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .withColumn("model_version", lit("v1.0")) \
    .withColumn("source_model", lit("silver_hotel_bookings"))

print(f"✅ fact_booking created: {fact_booking.count():,} bookings")

# DATA QUALITY TESTS for fact_booking
print("\n🧪 Running tests for fact_booking...")

# Test 1: booking_id should be unique
test1_duplicates = fact_booking.count() - fact_booking.select("booking_id").distinct().count()
if test1_duplicates == 0:
    print("   ✅ TEST PASSED: booking_id is unique")
else:
    print(f"   ❌ TEST FAILED: {test1_duplicates} duplicate booking_ids")

# Test 2: booking_id should not be null
test2_nulls = fact_booking.filter(col("booking_id").isNull()).count()
if test2_nulls == 0:
    print("   ✅ TEST PASSED: booking_id has no nulls")
else:
    print(f"   ❌ TEST FAILED: {test2_nulls} null booking_ids")

# Test 3: hotel_id should not be null
test3_nulls = fact_booking.filter(col("hotel_id").isNull()).count()
if test3_nulls == 0:
    print("   ✅ TEST PASSED: hotel_id has no nulls")
else:
    print(f"   ❌ TEST FAILED: {test3_nulls} null hotel_ids")

# Test 4: customer_id should not be null
test4_nulls = fact_booking.filter(col("customer_id").isNull()).count()
if test4_nulls == 0:
    print("   ✅ TEST PASSED: customer_id has no nulls")
else:
    print(f"   ❌ TEST FAILED: {test4_nulls} null customer_ids")

# Test 5: nights should be positive
test5_invalid = fact_booking.filter(
    col("nights").isNotNull() & (col("nights") <= 0)
).count()
if test5_invalid == 0:
    print("   ✅ TEST PASSED: all nights are positive")
else:
    print(f"   ⚠️  TEST WARNING: {test5_invalid} bookings with non-positive nights")

# Write to Gold
print("\n💾 Writing fact_booking to Gold layer...")
fact_booking.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("booking_year", "booking_month") \
    .saveAsTable("gold_fact_booking")

print("✅ fact_booking saved successfully (partitioned by booking_year, booking_month)")

# ============================================================================
# MODEL 4: KPI_REVENUE (Revenue KPIs)
# ============================================================================
print("\n" + "=" * 80)
print("📊 MODEL 4: KPI_REVENUE (Revenue Metrics)")
print("=" * 80)

print("\n🔨 Building kpi_revenue...")

# Aggregate revenue metrics by multiple dimensions
kpi_revenue = fact_booking.groupBy(
    "booking_year",
    "booking_month",
    "hotel_id",
    "booking_status",
    "payment_status"
).agg(
    count("booking_id").alias("total_bookings"),
    countDistinct("customer_id").alias("unique_customers"),
    _sum("total_price").alias("total_revenue"),
    avg("total_price").alias("avg_booking_value"),
    _sum("nights").alias("total_nights"),
    avg("nights").alias("avg_nights_per_booking"),
    _sum("total_guests").alias("total_guests"),
    avg("total_guests").alias("avg_guests_per_booking"),
    _sum(when(col("is_cancelled") == True, 1).otherwise(0)).alias("cancelled_bookings"),
    _sum(when(col("is_cancelled") == False, 1).otherwise(0)).alias("confirmed_bookings")
)

# Add calculated KPIs
kpi_revenue = kpi_revenue \
    .withColumn("cancellation_rate",
        spark_round((col("cancelled_bookings") / col("total_bookings") * 100), 2)
    ) \
    .withColumn("revenue_per_night",
        spark_round(col("total_revenue") / col("total_nights"), 2)
    ) \
    .withColumn("revenue_per_guest",
        spark_round(col("total_revenue") / col("total_guests"), 2)
    )

# Add Gold metadata
kpi_revenue = kpi_revenue \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .withColumn("model_version", lit("v1.0")) \
    .withColumn("source_model", lit("gold_fact_booking"))

print(f"✅ kpi_revenue created: {kpi_revenue.count():,} aggregated rows")

# DATA QUALITY TESTS for kpi_revenue
print("\n🧪 Running tests for kpi_revenue...")

# Test 1: total_bookings should be positive
test1_invalid = kpi_revenue.filter(col("total_bookings") <= 0).count()
if test1_invalid == 0:
    print("   ✅ TEST PASSED: all total_bookings are positive")
else:
    print(f"   ❌ TEST FAILED: {test1_invalid} rows with non-positive bookings")

# Test 2: total_revenue should not be null
test2_nulls = kpi_revenue.filter(col("total_revenue").isNull()).count()
if test2_nulls == 0:
    print("   ✅ TEST PASSED: total_revenue has no nulls")
else:
    print(f"   ❌ TEST FAILED: {test2_nulls} null total_revenue values")

# Test 3: cancellation_rate should be between 0-100
test3_invalid = kpi_revenue.filter(
    (col("cancellation_rate") < 0) | (col("cancellation_rate") > 100)
).count()
if test3_invalid == 0:
    print("   ✅ TEST PASSED: cancellation_rate is between 0-100")
else:
    print(f"   ❌ TEST FAILED: {test3_invalid} invalid cancellation rates")

# Show sample KPIs
print("\n📊 Sample KPI data:")
kpi_revenue.orderBy(col("total_revenue").desc()).show(5, truncate=False)

# Write to Gold
print("\n💾 Writing kpi_revenue to Gold layer...")
kpi_revenue.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_kpi_revenue")

print("✅ kpi_revenue saved successfully")

# ============================================================================
# BONUS MODEL 5: DIM_DATE (Date Dimension)
# ============================================================================
print("\n" + "=" * 80)
print("📊 BONUS MODEL 5: DIM_DATE (Date Dimension)")
print("=" * 80)

print("\n🔨 Building dim_date...")

# Extract all unique dates from bookings
all_dates = fact_booking.select("booking_date").union(
    fact_booking.select("checkin_date")
).union(
    fact_booking.select("checkout_date")
).distinct().filter(col("booking_date").isNotNull())

# Create date dimension
dim_date = all_dates.select(
    col("booking_date").alias("date"),
    year(col("booking_date")).alias("year"),
    month(col("booking_date")).alias("month"),
    dayofmonth(col("booking_date")).alias("day"),
    when(month(col("booking_date")).between(1, 3), "Q1")
        .when(month(col("booking_date")).between(4, 6), "Q2")
        .when(month(col("booking_date")).between(7, 9), "Q3")
        .otherwise("Q4").alias("quarter"),
    when(month(col("booking_date")).isin(12, 1, 2), "Winter")
        .when(month(col("booking_date")).isin(3, 4, 5), "Spring")
        .when(month(col("booking_date")).isin(6, 7, 8), "Summer")
        .otherwise("Fall").alias("season")
)

# Add metadata
dim_date = dim_date \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .withColumn("model_version", lit("v1.0"))

print(f"✅ dim_date created: {dim_date.count():,} unique dates")

# Write to Gold
dim_date.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_dim_date")

print("✅ dim_date saved successfully")

# ============================================================================
# DOCUMENTATION & LINEAGE
# ============================================================================
print("\n" + "=" * 80)
print("📚 MODEL DOCUMENTATION & LINEAGE")
print("=" * 80)

documentation = {
    "gold_dim_hotel": {
        "description": "Hotel dimension table containing unique hotel information",
        "primary_key": "hotel_id",
        "source": "silver_hotel_bookings",
        "tests": ["unique(hotel_id)", "not_null(hotel_id)", "not_null(hotel_name)"],
        "columns": ["hotel_id", "hotel_name", "country", "city", "hotel_type", "star_rating"]
    },
    "gold_dim_customer": {
        "description": "Customer dimension table with demographic information",
        "primary_key": "customer_id",
        "source": "silver_hotel_bookings",
        "tests": ["unique(customer_id)", "not_null(customer_id)", "email_format(email)"],
        "columns": ["customer_id", "first_name", "last_name", "email", "customer_age"]
    },
    "gold_fact_booking": {
        "description": "Fact table containing all booking transactions with metrics",
        "primary_key": "booking_id",
        "foreign_keys": ["hotel_id", "customer_id"],
        "source": "silver_hotel_bookings",
        "tests": ["unique(booking_id)", "not_null(booking_id)", "not_null(hotel_id)", "not_null(customer_id)"],
        "partitioned_by": ["booking_year", "booking_month"],
        "metrics": ["total_price", "nights", "total_guests", "revenue_per_night"]
    },
    "gold_kpi_revenue": {
        "description": "Aggregated revenue KPIs by time period, hotel, and status",
        "grain": "booking_year, booking_month, hotel_id, booking_status, payment_status",
        "source": "gold_fact_booking",
        "tests": ["positive(total_bookings)", "not_null(total_revenue)", "range(cancellation_rate, 0, 100)"],
        "metrics": ["total_revenue", "avg_booking_value", "cancellation_rate", "revenue_per_night"]
    },
    "gold_dim_date": {
        "description": "Date dimension for time-based analysis",
        "primary_key": "date",
        "source": "gold_fact_booking",
        "columns": ["date", "year", "month", "quarter", "season"]
    }
}

print("\n📋 Model Documentation:")
for model_name, doc in documentation.items():
    print(f"\n🏷️  {model_name}:")
    print(f"   Description: {doc['description']}")
    print(f"   Source: {doc['source']}")
    if 'tests' in doc:
        print(f"   Tests: {', '.join(doc['tests'])}")

print("\n🔗 Data Lineage:")
print("   silver_hotel_bookings")
print("   ├── gold_dim_hotel")
print("   ├── gold_dim_customer")
print("   ├── gold_fact_booking")
print("   │   ├── gold_kpi_revenue")
print("   │   └── gold_dim_date")
print("   silver_weather_data (available for enrichment)")
print("   silver_currency_rates (available for enrichment)")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("\n" + "=" * 80)
print("✅ GOLD LAYER MODELS COMPLETED!")
print("=" * 80)

# Get final counts
dim_hotel_count = spark.read.table("gold_dim_hotel").count()
dim_customer_count = spark.read.table("gold_dim_customer").count()
fact_booking_count = spark.read.table("gold_fact_booking").count()
kpi_revenue_count = spark.read.table("gold_kpi_revenue").count()
dim_date_count = spark.read.table("gold_dim_date").count()

print(f"\n📊 Gold Layer Summary:")
print(f"   • gold_dim_hotel: {dim_hotel_count:,} hotels")
print(f"   • gold_dim_customer: {dim_customer_count:,} customers")
print(f"   • gold_fact_booking: {fact_booking_count:,} bookings")
print(f"   • gold_kpi_revenue: {kpi_revenue_count:,} aggregated KPIs")
print(f"   • gold_dim_date: {dim_date_count:,} dates")

print(f"\n✅ All models include:")
print(f"   • Data quality tests (unique, not_null, custom)")
print(f"   • Documentation (description, source, columns)")
print(f"   • Lineage tracking (source models)")
print(f"   • Metadata (timestamps, versions)")

print(f"\n🎯 Gold layer ready for analytics and Power BI!")
print("=" * 80)


StatementMeta(, f5cc5bec-e06b-48a3-bdf8-48b73a7cedcb, 3, Finished, Available, Finished)

🏆 GOLD LAYER - DBT BUSINESS MODELS

📂 Reading Silver tables...
✅ Silver bookings: 839,597 records
✅ Weather data: 6 records
✅ Currency data: 166 records

📊 MODEL 1: DIM_HOTEL (Hotel Dimension)

🔨 Building dim_hotel...
✅ dim_hotel created: 839,597 unique hotels

🧪 Running tests for dim_hotel...
   ❌ TEST FAILED: 836657 duplicate hotel_ids
   ✅ TEST PASSED: hotel_id has no nulls
   ❌ TEST FAILED: 48656 null hotel_names

💾 Writing dim_hotel to Gold layer...
✅ dim_hotel saved successfully

📊 MODEL 2: DIM_CUSTOMER (Customer Dimension)

🔨 Building dim_customer...
✅ dim_customer created: 839,597 unique customers

🧪 Running tests for dim_customer...
   ❌ TEST FAILED: 829419 duplicate customer_ids
   ✅ TEST PASSED: customer_id has no nulls
   ✅ TEST PASSED: all emails are valid format

💾 Writing dim_customer to Gold layer...
✅ dim_customer saved successfully

📊 MODEL 3: FACT_BOOKING (Fact Table)

🔨 Building fact_booking...
✅ fact_booking created: 839,597 bookings

🧪 Running tests for fact_booki

In [2]:
# CRITICAL FIX: Remove ALL duplicates from dimensions
print("🔧 CRITICAL FIX - Removing ALL duplicates")
print("=" * 80)

from pyspark.sql.functions import row_number, col, current_timestamp, lit, year, when
from pyspark.sql.window import Window
from pyspark.sql.functions import first


df_silver = spark.read.table("silver_hotel_bookings")

# ============================================================================
# FIX 1: dim_hotel - Keep only TRULY unique hotels
# ============================================================================
print("\n🏨 Fixing dim_hotel - removing ALL duplicates...")

# Group by hotel_id and aggregate to get one record per hotel
dim_hotel_clean = df_silver.groupBy("hotel_id").agg(
    first("hotel_name").alias("hotel_name"),
    first("country").alias("country"),
    first("city").alias("city"),
    first("hotel_type").alias("hotel_type"),
    first("star_rating").alias("star_rating"),
    first("total_rooms").alias("total_rooms"),
    first("hotel_facilities").alias("hotel_facilities"),
    first("latitude").alias("latitude"),
    first("longitude").alias("longitude")
).filter(col("hotel_id").isNotNull())

# Add metadata
dim_hotel_clean = dim_hotel_clean \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .withColumn("model_version", lit("v1.2")) \
    .withColumn("source_model", lit("silver_hotel_bookings"))

# Test uniqueness
hotel_count = dim_hotel_clean.count()
distinct_hotel_count = dim_hotel_clean.select("hotel_id").distinct().count()

if hotel_count == distinct_hotel_count:
    print(f"✅ dim_hotel is NOW unique: {hotel_count:,} hotels")
else:
    print(f"❌ STILL has duplicates: {hotel_count:,} vs {distinct_hotel_count:,}")

# Write
dim_hotel_clean.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold_dim_hotel")

# ============================================================================
# FIX 2: dim_customer - Keep only TRULY unique customers
# ============================================================================
print("\n👤 Fixing dim_customer - removing ALL duplicates...")

# Group by customer_id and aggregate
dim_customer_clean = df_silver.groupBy("customer_id").agg(
    first("first_name").alias("first_name"),
    first("last_name").alias("last_name"),
    first("full_name").alias("full_name"),
    first("email").alias("email"),
    first("phone").alias("phone"),
    first("birth_date").alias("birth_date"),
    first("gender").alias("gender"),
    first("country_customer").alias("country"),
    first("city_customer").alias("city"),
    first("language_preference").alias("language_preference"),
    first("loyalty_level_customer").alias("loyalty_level"),
    first("registration_date").alias("registration_date"),
    first("marketing_consent").alias("marketing_consent")
).filter(col("customer_id").isNotNull())

# Add derived columns
dim_customer_clean = dim_customer_clean \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .withColumn("model_version", lit("v1.2")) \
    .withColumn("source_model", lit("silver_hotel_bookings")) \
    .withColumn("customer_age", 
        when(col("birth_date").isNotNull(), 
             year(current_timestamp()) - year(col("birth_date"))
        ).otherwise(None)
    )

# Test uniqueness
customer_count = dim_customer_clean.count()
distinct_customer_count = dim_customer_clean.select("customer_id").distinct().count()

if customer_count == distinct_customer_count:
    print(f"✅ dim_customer is NOW unique: {customer_count:,} customers")
else:
    print(f"❌ STILL has duplicates: {customer_count:,} vs {distinct_customer_count:,}")

# Write
dim_customer_clean.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold_dim_customer")

print("\n🎉 Dimensions fixed with aggregation!")
print("=" * 80)

StatementMeta(, f5cc5bec-e06b-48a3-bdf8-48b73a7cedcb, 4, Finished, Available, Finished)

🔧 CRITICAL FIX - Removing ALL duplicates

🏨 Fixing dim_hotel - removing ALL duplicates...
✅ dim_hotel is NOW unique: 2,940 hotels

👤 Fixing dim_customer - removing ALL duplicates...
✅ dim_customer is NOW unique: 10,178 customers

🎉 Dimensions fixed with aggregation!


In [3]:
# FIX: Dimension Tables - Proper Deduplication
print("🔧 FIXING DIMENSION TABLES")
print("=" * 80)

# Fix dim_hotel - deduplicate by hotel_id properly
print("\n🏨 Fixing dim_hotel...")
df_silver = spark.read.table("silver_hotel_bookings")

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Create window to get one record per hotel_id
window_hotel = Window.partitionBy("hotel_id").orderBy(col("hotel_name").desc())

dim_hotel_fixed = df_silver.select(
    "hotel_id", "hotel_name", "country", "city", "hotel_type",
    "star_rating", "total_rooms", "hotel_facilities", "latitude", "longitude"
).withColumn("row_num", row_number().over(window_hotel)) \
 .filter(col("row_num") == 1) \
 .drop("row_num") \
 .withColumn("gold_ingestion_timestamp", current_timestamp()) \
 .withColumn("model_version", lit("v1.1")) \
 .withColumn("source_model", lit("silver_hotel_bookings"))

dim_hotel_fixed.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold_dim_hotel")
hotel_count = spark.read.table("gold_dim_hotel").count()
print(f"✅ dim_hotel fixed: {hotel_count:,} unique hotels")

# Fix dim_customer - deduplicate by customer_id properly
print("\n👤 Fixing dim_customer...")
window_customer = Window.partitionBy("customer_id").orderBy(col("registration_date").desc())

dim_customer_fixed = df_silver.select(
    "customer_id", "first_name", "last_name", "full_name", "email", "phone",
    "birth_date", "gender", "country_customer", "city_customer",
    "language_preference", "loyalty_level_customer", "registration_date", "marketing_consent"
).withColumn("row_num", row_number().over(window_customer)) \
 .filter(col("row_num") == 1) \
 .drop("row_num") \
 .withColumn("gold_ingestion_timestamp", current_timestamp()) \
 .withColumn("model_version", lit("v1.1")) \
 .withColumn("source_model", lit("silver_hotel_bookings")) \
 .withColumn("customer_age", 
    when(col("birth_date").isNotNull(), year(current_timestamp()) - year(col("birth_date"))).otherwise(None)
)

dim_customer_fixed.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold_dim_customer")
customer_count = spark.read.table("gold_dim_customer").count()
print(f"✅ dim_customer fixed: {customer_count:,} unique customers")

print("\n🎉 Dimensions fixed!")

StatementMeta(, f5cc5bec-e06b-48a3-bdf8-48b73a7cedcb, 5, Finished, Available, Finished)

🔧 FIXING DIMENSION TABLES

🏨 Fixing dim_hotel...
✅ dim_hotel fixed: 2,940 unique hotels

👤 Fixing dim_customer...
✅ dim_customer fixed: 10,178 unique customers

🎉 Dimensions fixed!


In [4]:
# ============================================================================
# MODEL 2.5: DIM_CITY (Şehir Boyutu - DÜZELTİLMİŞ)
# ============================================================================
print("\n" + "=" * 80)
print("📊 MODEL 2.5: DIM_CITY (Şehir Boyutu)")
print("=" * 80)

print("\n🔨 Building dim_city...")

# Otel ve Müşteri verilerinden benzersiz şehirleri çekin
# gold_dim_hotel'deki sütun adları: city ve country (Doğru)
hotel_cities = spark.read.table("gold_dim_hotel").select( 
    col("city").alias("city_name"), 
    col("country").alias("country_name")
)

# gold_dim_customer'daki gerçek sütun adları: city_customer ve country_customer (Düzeltildi)
customer_cities = spark.read.table("gold_dim_customer").select(
    col("city_customer").alias("city_name"), 
    col("country_customer").alias("country_name")
)

# Tüm benzersiz şehirleri birleştirin (unionByName ile şema uyumu sağlandı)
dim_city = hotel_cities.unionByName(customer_cities).distinct().filter(col("city_name").isNotNull())

# Şehirlere benzersiz bir ID atayın (city_id)
window_spec = Window.orderBy("country_name", "city_name")
dim_city = dim_city.withColumn("city_id", row_number().over(window_spec))

# Add metadata
dim_city = dim_city \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .withColumn("model_version", lit("v1.0")) \
    .withColumn("source_model", lit("gold_dim_hotel, gold_dim_customer"))

print(f"✅ dim_city created: {dim_city.count():,} unique cities")

# DATA QUALITY TESTS for dim_city
print("\n🧪 Running tests for dim_city...")
test1_duplicates = dim_city.count() - dim_city.select("city_id").distinct().count()
print(f"   {'✅ TEST PASSED' if test1_duplicates == 0 else f'❌ TEST FAILED: {test1_duplicates} duplicates'}: city_id is unique")
test2_nulls = dim_city.filter(col("city_name").isNull()).count()
print(f"   {'✅ TEST PASSED' if test2_nulls == 0 else f'❌ TEST FAILED: {test2_nulls} nulls'}: city_name has no nulls")

# Write to Gold
print("\n💾 Writing dim_city to Gold layer...")
dim_city.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_dim_city")

print("✅ gold_dim_city saved successfully")


# ============================================================================
# DOCUMENTATION & LINEAGE - GÜNCELLENMİŞ
# ============================================================================
print("\n" + "=" * 80)
print("📚 MODEL DOCUMENTATION & LINEAGE (Güncellenmiş)")
print("=" * 80)

# Artık tüm modelleri ve doğru testleri/metrikleri içeriyor
documentation = {
    "gold_dim_hotel": {
        "description": "Hotel dimension table containing unique hotel information (Critically fixed using Group By aggregation).",
        "primary_key": "hotel_id",
        "source": "silver_hotel_bookings",
        "tests": ["unique(hotel_id)", "not_null(hotel_id)", "not_null(hotel_name)"],
        "columns": ["hotel_id", "hotel_name", "country", "city", "hotel_type", "star_rating"]
    },
    "gold_dim_customer": {
        "description": "Customer dimension table with demographic information (Critically fixed using Group By aggregation).",
        "primary_key": "customer_id",
        "source": "silver_hotel_bookings",
        "tests": ["unique(customer_id)", "not_null(customer_id)", "email_format(email)"],
        "columns": ["customer_id", "first_name", "last_name", "email", "customer_age"]
    },
    "gold_dim_city": { 
        "description": "Unique cities and countries derived from both hotel and customer locations for geographic analysis.",
        "primary_key": "city_id",
        "source": "gold_dim_hotel, gold_dim_customer",
        "tests": ["unique(city_id)", "not_null(city_name)"],
        "columns": ["city_id", "city_name", "country_name"]
    },
    "gold_fact_booking": {
        "description": "Fact table containing all booking transactions and calculated metrics.",
        "primary_key": "booking_id",
        "foreign_keys": ["hotel_id", "customer_id"],
        "source": "silver_hotel_bookings",
        "tests": ["unique(booking_id)", "not_null(booking_id)", "not_null(hotel_id)", "not_null(customer_id)", "positive(nights)"],
        "partitioned_by": ["booking_year", "booking_month"],
        "metrics": ["total_price", "nights", "total_guests", "revenue_per_night", "lead_time_days"]
    },
    "gold_kpi_revenue": {
        "description": "Aggregated revenue KPIs by time period, hotel, and status.",
        "grain": "booking_year, booking_month, hotel_id, booking_status, payment_status",
        "source": "gold_fact_booking",
        "tests": ["positive(total_bookings)", "not_null(total_revenue)", "range(cancellation_rate, 0, 100)"],
        "metrics": ["total_revenue", "avg_booking_value", "cancellation_rate", "revenue_per_night", "revenue_per_guest"]
    },
    "gold_dim_date": {
        "description": "Date dimension for time-based analysis.",
        "primary_key": "date",
        "source": "gold_fact_booking",
        "columns": ["date", "year", "month", "quarter", "season"]
    }
}

print("\n📋 Model Documentation:")
for model_name, doc in documentation.items():
    print(f"\n🏷️  {model_name}:")
    print(f"   Description: {doc['description']}")
    print(f"   Source: {doc['source']}")
    if 'tests' in doc:
        print(f"   Tests: {', '.join(doc['tests'])}")

print("\n🔗 Data Lineage:")
print("   silver_hotel_bookings")
print("   ├── gold_dim_hotel")
print("   ├── gold_dim_customer")
print("   ├── gold_dim_city") 
print("   ├── gold_fact_booking")
print("   │   ├── gold_kpi_revenue")
print("   │   └── gold_dim_date")
print("   silver_weather_data (available for enrichment)")
print("   silver_currency_rates (available for enrichment)")

# ============================================================================
# FINAL SUMMARY TEKRAR ÇALIŞTIRMA (Opsiyonel)
# ============================================================================
print("\n" + "=" * 80)
print("✅ GOLD LAYER MODELLERİ TAMAMLANDI!")
print("=" * 80)

# Son sayımları almak için tekrar okuma
try:
    dim_hotel_count = spark.read.table("gold_dim_hotel").count()
    dim_customer_count = spark.read.table("gold_dim_customer").count()
    fact_booking_count = spark.read.table("gold_fact_booking").count()
    kpi_revenue_count = spark.read.table("gold_kpi_revenue").count()
    dim_date_count = spark.read.table("gold_dim_date").count()
    dim_city_count = spark.read.table("gold_dim_city").count() 

    print(f"\n📊 Gold Layer Summary:")
    print(f"   • gold_dim_hotel: {dim_hotel_count:,} hotels")
    print(f"   • gold_dim_customer: {dim_customer_count:,} customers")
    print(f"   • gold_dim_city: {dim_city_count:,} cities") 
    print(f"   • gold_fact_booking: {fact_booking_count:,} bookings")
    print(f"   • gold_kpi_revenue: {kpi_revenue_count:,} aggregated KPIs")
    print(f"   • gold_dim_date: {dim_date_count:,} dates")

except Exception as e:
    print(f"\n❌ Final Summary hatası: Tabloların hepsi henüz kaydedilmemiş olabilir. Hata: {e}")

StatementMeta(, f5cc5bec-e06b-48a3-bdf8-48b73a7cedcb, 6, Finished, Available, Finished)


📊 MODEL 2.5: DIM_CITY (Şehir Boyutu)

🔨 Building dim_city...
✅ dim_city created: 731 unique cities

🧪 Running tests for dim_city...
   ✅ TEST PASSED: city_id is unique
   ✅ TEST PASSED: city_name has no nulls

💾 Writing dim_city to Gold layer...
✅ gold_dim_city saved successfully

📚 MODEL DOCUMENTATION & LINEAGE (Güncellenmiş)

📋 Model Documentation:

🏷️  gold_dim_hotel:
   Description: Hotel dimension table containing unique hotel information (Critically fixed using Group By aggregation).
   Source: silver_hotel_bookings
   Tests: unique(hotel_id), not_null(hotel_id), not_null(hotel_name)

🏷️  gold_dim_customer:
   Description: Customer dimension table with demographic information (Critically fixed using Group By aggregation).
   Source: silver_hotel_bookings
   Tests: unique(customer_id), not_null(customer_id), email_format(email)

🏷️  gold_dim_city:
   Description: Unique cities and countries derived from both hotel and customer locations for geographic analysis.
   Source: gold_di

In [5]:
# ============================================================================
# GOLD FACT BOOKING - CITY COLUMN EKLEME
# ============================================================================

print("="*80)
print("UPDATING gold_fact_booking WITH CITY")
print("="*80)

# Mevcut fact tablosunu oku
df_fact_current = spark.table("bronze_lakehouse.gold_fact_booking")

print(f"Current fact records: {df_fact_current.count():,}")

# Silver'dan city bilgisini getir
df_silver = spark.table("bronze_lakehouse.silver_hotel_bookings")

# Fact'e city ekle (booking_id ile join)
df_fact_with_city = df_fact_current.alias("fact").join(
    df_silver.select("booking_id", "city").alias("silver"),
    col("fact.booking_id") == col("silver.booking_id"),
    "left"
).select("fact.*", col("silver.city").alias("city"))

print(f"Fact with city: {df_fact_with_city.count():,}")

# City null kontrolü
null_cities = df_fact_with_city.filter(col("city").isNull()).count()
print(f"Records without city: {null_cities:,}")

# Güncellenmiş tabloyu yaz
df_fact_with_city.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("bronze_lakehouse.gold_fact_booking")

print("gold_fact_booking updated with city column")

# Doğrulama
print("\nSample with city:")
spark.table("bronze_lakehouse.gold_fact_booking").select(
    "booking_id", "hotel_id", "customer_id", "city", "total_price"
).show(10, truncate=False)

StatementMeta(, f5cc5bec-e06b-48a3-bdf8-48b73a7cedcb, 7, Finished, Available, Finished)

UPDATING gold_fact_booking WITH CITY
Current fact records: 839,597
Fact with city: 839,597
Records without city: 49,006
gold_fact_booking updated with city column

Sample with city:
+-------------+--------+-----------+---------+-----------+
|booking_id   |hotel_id|customer_id|city     |total_price|
+-------------+--------+-----------+---------+-----------+
|BKG_000000012|H00207  |C0003533   |Rotterdam|566.83     |
|BKG_000000019|H00951  |C0001231   |Bangkok  |1071.29    |
|BKG_000000061|H00753  |C0001698   |Medellín |1754.26    |
|BKG_000000064|H00279  |C0004108   |Mombasa  |1976.14    |
|BKG_000000115|H00926  |C0000155   |Plovdiv  |877.23     |
|BKG_000000117|H00521  |C0000032   |Uppsala  |720.7      |
|BKG_000000128|H00096  |C0002194   |Busan    |NULL       |
|BKG_000000152|H00138  |C0000723   |NULL     |347.23     |
|BKG_000000166|H00228  |C0004043   |Eindhoven|1187.33    |
|BKG_000000251|H00362  |C0002854   |Singapore|486.36     |
+-------------+--------+-----------+---------+-----

In [6]:
# ============================================================================
# CRITICAL FIX: FACT_BOOKING'e CITY_ID EKLEME
# ============================================================================
print("\n" + "=" * 80)
print("🔧 FACT_BOOKING FIX: CITY ID EKLEME VE ESKİ İLİŞKİYİ DÜZELTME")
print("=" * 80)

from pyspark.sql.functions import col, row_number, coalesce, lit
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

# 1. Kaynak Tabloları Oku
df_silver = spark.read.table("silver_hotel_bookings")
df_dim_city = spark.read.table("gold_dim_city") # Artık CITY_ID içeriyor

# 2. Fact tablosuna Join yapabilmek için Silver'dan booking_id, country ve city verilerini al
fact_booking_join_data = df_silver.filter(
    col("booking_id").isNotNull()
).select(
    col("booking_id"),
    col("country").alias("join_country"), # Otel ülkesi
    col("city").alias("join_city")       # Otel şehri
).distinct()

# 3. Fact Join Data ile DIM_CITY'yi JOIN ederek CITY_ID'yi Bul
fact_booking_with_id = fact_booking_join_data.join(
    df_dim_city.select("city_id", col("city_name"), col("country_name")),
    (col("join_city") == col("city_name")) & (col("join_country") == col("country_name")),
    "left"
).select(
    col("booking_id"),
    col("city_id") # Yeni Foreign Key'imiz!
)

# 4. Ana Fact Tablosunu Oku ve Yeni CITY ID Sütununu Ekle
df_fact = spark.read.table("gold_fact_booking")

# Booking ID üzerinden join yaparak CITY ID'yi Fact'e ekle
# Eski hatalı 'city' sütununu da tutmaya devam ediyoruz (denormalizasyon için)
df_fact_final = df_fact.drop("city_id") \
                       .join(
                           fact_booking_with_id,
                           "booking_id",
                           "left"
                       )

# 5. Gold Tablosuna Kaydet
print(f"✅ gold_fact_booking güncelleniyor. Kayıt sayısı: {df_fact_final.count():,}")

df_fact_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_fact_booking")

print("✅ gold_fact_booking tablosu CITY_ID ile başarıyla güncellendi.")

StatementMeta(, f5cc5bec-e06b-48a3-bdf8-48b73a7cedcb, 8, Finished, Available, Finished)


🔧 FACT_BOOKING FIX: CITY ID EKLEME VE ESKİ İLİŞKİYİ DÜZELTME
✅ gold_fact_booking güncelleniyor. Kayıt sayısı: 839,597
✅ gold_fact_booking tablosu CITY_ID ile başarıyla güncellendi.
